[Link repository del progetto 6](https://github.com/FTocci/TGW2D)

Maraziti Matteo 534932

Tocci Federico 533449

Scordino Giacomo 533393

**STUDIO PRELIMINARE**

In [12]:
using ViewerGL
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using SparseArrays

include("../LinearAlgebraicRepresentation.jl-master/src/arrangement/planar_arrangement.jl")

V = [2 2; 1.5 2; 3 3.5; 1 3; 5 3; 1 2; 5 2]

EV = SparseArrays.sparse(Array{Int8, 2}([
               [1 1 0 0 0 0 0] #1->1,2
               [0 1 1 0 0 0 0] #2->2,3
               [1 0 1 0 0 0 0] #3->1,3
               [0 0 0 1 1 0 0] #4->4,5
               [0 0 0 0 0 1 1] #5->6,7
           ]))

sigma=spzeros(Int8, 0)

return_edge_map=false

multiproc=false

W = convert(Lar.Points,V')

V, copEV, copFE=planar_arrangement(V,EV,sigma,return_edge_map,multiproc)

EV = Lar.cop2lar(copEV)

FE = Lar.cop2lar(copFE)

FV=EV

FV = convert(Lar.Cells,FV)

VV = [[k] for k=1:size(V,2)]

model = (W, [VV,EV,FV])::Lar.LARmodel;

meshes = GL.numbering(1.5)(model, GL.COLORS[1], 0.1);

GL.VIEW(meshes);

Le operazioni sopra presentate sono state utilizzate per la realizzazione dei primi test.  L'obiettivo e' stato quello di verificare il corretto funzionamento dell'algoritmo e di individuare i moduli di interesse.   Con l'utilizzo di ViewerGL è stato possibile visualizzare graficamente il piano partizionato individuato dai parametri scelti.

[RisultatoTest](/images/RisultatoTest.PNG)

--------------------------------------

**STUDIO ESECUTIVO**

Esecuzione classe di test prima di apportare modifiche(parallelizzazione del codice e rifattorizzazione).

In [46]:
@time include("../LinearAlgebraicRepresentation.jl-master/test/planar_arrangement.jl")

Test Summary:            | Pass  Total
Edge fragmentation tests |    3      3
Test Summary:           | Pass  Total
merge_vertices test set |    2      2
Test Summary:                   | Pass  Total
biconnected_components test set |    1      1
Test Summary: | Pass  Total
Face creation |    5      5
  3.132530 seconds (1.71 M allocations: 94.434 MiB, 2.15% gc time, 85.78% compilation time)


Test.DefaultTestSet("Face creation", Any[Test.DefaultTestSet("External cell individuation", Any[], 1, false, false), Test.DefaultTestSet("Containment test", Any[], 2, false, false), Test.DefaultTestSet("Transitive reduction", Any[], 1, false, false), Test.DefaultTestSet("Cell merging", Any[], 1, false, false)], 0, false, false)

Esecuzione post-modifiche, parallelizzate porzioni di codice e rifattorizzazione

In [48]:
@time include("../LinearAlgebraicRepresentation.jl-master/test/planar_arrangement.jl")

Test Summary:            | Pass  Total
Edge fragmentation tests |    3      3
Test Summary:           | Pass  Total
merge_vertices test set |    2      2
Test Summary:                   | Pass  Total
biconnected_components test set |    1      1
Test Summary: | Pass  Total
Face creation |    5      5
  2.275163 seconds (1.71 M allocations: 94.430 MiB, 85.12% compilation time)


Test.DefaultTestSet("Face creation", Any[Test.DefaultTestSet("External cell individuation", Any[], 1, false, false), Test.DefaultTestSet("Containment test", Any[], 2, false, false), Test.DefaultTestSet("Transitive reduction", Any[], 1, false, false), Test.DefaultTestSet("Cell merging", Any[], 1, false, false)], 0, false, false)

*Risparmio temporale del 30%*

------------------------------------------